Each time the qubit is rotated and measured, it has 50/50 chance of returning 0 or 1. Tracing through the stacktrace of the PRNG calls, we see that qiskit uses `std::mt19937_64` with our seed and extracts information using `std::discrete_distribution`, which in a 50/50 probability distribution is as good as extracting MSB of calls to `rng()` aka `std::mt19937_64::operator()`.

The stacktrace to RNG calls looks like:

- https://github.com/Qiskit/qiskit-aer/blob/9999dfbe3a6e39a222fa3477888f0c02f7a178b5/src/controllers/controller_execute.hpp#L155C1-L163
- https://github.com/Qiskit/qiskit-aer/blob/9999dfbe3a6e39a222fa3477888f0c02f7a178b5/src/framework/rng.hpp#L54C1-L58
- https://github.com/Qiskit/qiskit-aer/blob/9999dfbe3a6e39a222fa3477888f0c02f7a178b5/src/simulators/circuit_executor.hpp#L512-L532
- https://github.com/Qiskit/qiskit-aer/blob/9999dfbe3a6e39a222fa3477888f0c02f7a178b5/src/simulators/circuit_executor.hpp#L608C11-L608C28 i think? since shots=1
- https://github.com/Qiskit/qiskit-aer/blob/9999dfbe3a6e39a222fa3477888f0c02f7a178b5/src/simulators/circuit_executor.hpp#L758-L775
- https://github.com/Qiskit/qiskit-aer/blob/9999dfbe3a6e39a222fa3477888f0c02f7a178b5/src/simulators/state.hpp#L318C1-L318C1
- https://github.com/Qiskit/qiskit-aer/blob/9999dfbe3a6e39a222fa3477888f0c02f7a178b5/src/simulators/statevector/statevector_state.hpp#L514
- https://github.com/Qiskit/qiskit-aer/blob/9999dfbe3a6e39a222fa3477888f0c02f7a178b5/src/simulators/statevector/statevector_state.hpp#L946
- https://github.com/Qiskit/qiskit-aer/blob/9999dfbe3a6e39a222fa3477888f0c02f7a178b5/src/simulators/statevector/statevector_state.hpp#L972
- https://github.com/Qiskit/qiskit-aer/blob/9999dfbe3a6e39a222fa3477888f0c02f7a178b5/src/framework/rng.hpp#L95

We can verify that this is correct by calling `mt19937_64` with same seed seperately and observing the outputs.

In [13]:
def gen_otp(seed, n):
    from qiskit import QuantumCircuit, QuantumRegister, Aer, ClassicalRegister
    
    qr = QuantumRegister(1)
    cr = ClassicalRegister(n)
    qc = QuantumCircuit(qr, cr)
    for i in range(n):
        # Apply H-gate
        qc.h(0)
        # Measure the qubit
        qc.measure(0, i)

    # seed = int.from_bytes(os.urandom(8), 'big') & 0x7FFFFFFFFFFFFFFF
    sv_sim = Aer.get_backend('qasm_simulator')
    job = sv_sim.run(qc, seed_simulator=seed, shots=1)
    job_result = job.result()
    otp = list(job_result.get_counts().keys())[0]

    return otp

# qiskit will output first random state at the last bit, we reverse to 
# make comparison easier
gen_otp(1116, 64)[::-1] 

'0000000001010010101110010101110010101011100011010110110001110101'

In [14]:
cppsrc = """
#include <bits/stdc++.h>
using namespace std;

typedef unsigned long long int ulli;

int main() {
    mt19937_64 rng1, rng2;
    ulli seed = 1116; // same seed
    ulli n = 64;
    rng1.seed(seed);
    rng2.seed(seed);
    vector<int> probs = {1, 1};
    auto distrib = discrete_distribution(probs.begin(), probs.end());

    for (int i = 0; i < n; i++) {
        ulli res1 = distrib(rng1);
        printf("%llu", res1);
    }
    printf(" - from distribution\\n");

    for (int i = 0; i < n; i++) {
        ulli res2 = rng2();
        printf("%llu", res2 >> 63);
    }
    printf(" - from msb\\n");

    return 0;
}
"""

with open("gen.cpp", "w") as f:
    f.write(cppsrc)

# bruh my tensorflow install wrecks my glibc
%set_env LD_LIBRARY_PATH=
!g++ gen.cpp -o gen && chmod +x gen && ./gen

env: LD_LIBRARY_PATH=
0000000001010010101110010101110010101011100011010110110001110101 - from distribution
0000000001010010101110010101110010101011100011010110110001110101 - from msb


All bits of the twister are F2 linear, so a system which extracts only MSB of the twister is also F2 linear. We can model this as a LFSR with unknown taps and this will even work with the bitstream in reverse. This can be efficiently solved using some gaussian elimination and by modelling each row of the matrix as a 19937-bit integer, with `mpz` for speedup. Solver library has been placed in `libquantum.py` for reuse in part 2.

In [15]:
from gmpy2 import mpz
from tqdm.auto import tqdm
from libquantum import gaussian_elimination

In [16]:
otp = gen_otp(1116, 312 * 64 * 3)

In [17]:
state_bits = 19937

relations = {}
solutions = {}
for i in tqdm(range(state_bits)):
    relations[i] = mpz(int(otp[i:state_bits+i][::-1], 2))
    solutions[i] = int(otp[state_bits+i])

tap_coeffs = gaussian_elimination(relations, solutions)

  0%|          | 0/19937 [00:00<?, ?it/s]

  0%|          | 0/19937 [00:00<?, ?it/s]

  0%|          | 0/19937 [00:00<?, ?it/s]

In [18]:
taps = []
for pos, soln in tap_coeffs.items():
    if soln:
        taps.append(pos)
print(f"{len(taps)=}")
print(f"{taps=}")

len(taps)=284
taps=[0, 311, 467, 623, 779, 935, 1091, 1244, 1247, 1403, 1559, 1715, 1866, 1868, 1871, 2027, 2177, 2178, 2183, 2333, 2339, 2488, 2492, 2495, 2651, 2799, 2807, 2955, 2963, 3111, 3114, 3116, 3119, 3267, 3275, 3423, 3425, 3426, 3431, 3579, 3581, 3587, 3732, 3735, 3736, 3740, 3743, 3891, 3899, 4043, 4055, 4199, 4211, 4355, 4356, 4362, 4364, 4367, 4511, 4523, 4673, 4674, 4679, 4829, 4835, 4984, 4988, 4991, 5147, 5295, 5303, 5451, 5459, 5598, 5607, 5610, 5612, 5615, 5763, 5771, 5909, 5910, 5919, 5921, 5922, 5927, 6065, 6075, 6077, 6083, 6222, 6228, 6231, 6232, 6236, 6239, 6387, 6395, 6533, 6534, 6539, 6551, 6689, 6695, 6707, 6842, 6846, 6851, 6852, 6858, 6860, 6863, 7007, 7019, 7153, 7154, 7157, 7158, 7169, 7170, 7175, 7309, 7313, 7325, 7331, 7470, 7480, 7484, 7487, 7643, 7775, 7781, 7782, 7791, 7799, 7931, 7937, 7947, 7955, 8087, 8090, 8103, 8106, 8108, 8111, 8243, 8259, 8267, 8399, 8401, 8402, 8415, 8417, 8418, 8423, 8555, 8557, 8571, 8573, 8579, 8708, 8711, 8724, 8727, 8728

In [19]:
for pos in tqdm(range(30_000, 31_000)):
    history = otp[pos:state_bits+pos]
    goal = int(otp[state_bits+pos])

    cur = 0
    for tap in taps:
        if history[tap] == "1":
            cur ^= 1
    assert cur == goal

  0%|          | 0/1000 [00:00<?, ?it/s]